# "News Sentiment and Stock Market Data Exploration"
> "Performing Exploratory Data Analysis for the Saudi Stock Market"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [data-wrangling, EDA]
- image: images/arabic-stock-market-sentiment-analysis/featured_sectors_bar.png
- hide: false
- search_exclude: true

# News Sentiment and Stock Market Data Exploration

In [1]:
#!pip install pandas-profiling[notebook,html]
#!pip install cufflinks
#!pip install chart_studio
#!pip install googletrans

In [2]:
#plotly.offline doesn't push your charts to the clouds
import plotly.offline as pyo
#allows us to create the Data and Figure objects
from plotly.graph_objs import *
#plotly.plotly pushes your charts to the cloud  
# import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.express as px

import matplotlib.pyplot as plt
%matplotlib inline

# import cufflings to easily plot pandas data frames
import cufflinks as cf
# work with cufflinks offline and set its theme
import plotly.io as pio
pio.templates.default = "plotly_white"

cf.go_offline()
cf.set_config_file(theme='white')

import pandas as pd
from pandas_profiling import ProfileReport
from googletrans import Translator
import datetime as dt

import numpy as np

import os
import json
import re
import sys

In [3]:
print(pd.__version__)

1.0.3


In [4]:
def print_full(x):
    pd.set_option('display.max_colwidth', None)
    return x

def reset():
    pd.reset_option('display.max_colwidth')

In [5]:
cf.getThemes()

['ggplot', 'pearl', 'solar', 'space', 'white', 'polar', 'henanigans']

In [6]:
cf.colors.scales() 

In [7]:
df = pd.read_excel('Samira - Arabic Stock Market Data.xlsx')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34386 entries, 0 to 34385
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Sector                     34386 non-null  object        
 1   Company ID                 34386 non-null  int64         
 2   Company Name               34386 non-null  object        
 3   Date                       34386 non-null  object        
 4   Time                       34386 non-null  object        
 5   News Title                 34386 non-null  object        
 6   News Details               30098 non-null  object        
 7   Polarity                   34386 non-null  object        
 8   Polarity Words             34386 non-null  object        
 9   Date_1st_Additional        34386 non-null  datetime64[ns]
 10  Oppening                   34269 non-null  float64       
 11  The Highest                34269 non-null  float64       
 12  The 

In [9]:
df['Polarity'].value_counts()

Positive     15506
Neutral      11479
-             4288
Negative      3102
neutral          4
positive         3
negative         3
 Negative        1
Name: Polarity, dtype: int64

We need to normalize this column

In [10]:
df['Polarity'] = df['Polarity'].str.strip().str.capitalize()

In [11]:
df['Polarity'].value_counts()

Positive    15509
Neutral     11483
-            4288
Negative     3106
Name: Polarity, dtype: int64

In [12]:
df.head(2)

,Sector,Company ID,Company Name,Date,Time,News Title,News Details,Polarity,Polarity Words,Date_1st_Additional,Oppening,The Highest,The Lowest,Closing,Changing,Changing%,The quantity Handled,Total Current Value (SAR),Number of Deals
0,البنوك,1020,بنك الجزيرة,12/31/2011 12:00:00 AM,12:07:52,يعلن بنك الجزيرة عن توصية مجلس إدارته توزيع أر...,يعلن بنك الجزيرة عن توصية مجلس إدارته توزيع أر...,Positive,ارباح سنوية - توزيع أرباح,2011-12-31,9.07,9.12,8.93,8.96,0.11,1.24,1961798.0,17729676.1,514.0
1,البنوك,1050,البنك السعودي الفرنسي,12/30/2019 12:00:00 AM,15:22:03,يعلن البنك السعودي الفرنسي عن توصية مجلس الإدا...,بند توضيح مقدمة يعلن البنك السعودي الفرنسي ع...,Positive,توزيع - ارباح,2019-12-30,37.30,38.20,37.30,37.55,0.30,0.81,555205.0,20956971.9,634.0


In [13]:
translator = Translator()

unique_sectors = df['Sector'].unique().tolist()

unique_sectors_en = [translator.translate(sector, src='ar').text.title() for sector in unique_sectors]

print(unique_sectors)
print(unique_sectors_en)

['البنوك', 'مؤشر السوق الرئيسية (تاسي)', 'المواد الأساسية', 'السلع الرأسمالية', 'السلع طويلة الاجل', 'تجزئة السلع الكمالية', 'الخدمات الإستهلاكية', 'الخدمات التجارية والمهنية', 'الطاقة', 'إنتاج الأغذية', 'الادوية', 'المرافق العامة', 'الإستثمار والتمويل', 'النقل', 'الرعاية الصحية', 'تجزئة الأغذية', 'إدارة وتطوير العقارات', 'الإعلام والترفيه', 'الصناديق العقارية المتداولة', 'الإتصالات', 'التطبيقات وخدمات التقنية', 'التأمين']
['Banks', 'Main Market Index (Tasi)', 'Basic Materials', 'Capital Goods', 'Long-Term Goods', 'Retail Of Luxury Goods', 'Consumer Services', 'Business And Professional Services', 'Energy', 'Food Production', 'Pharmaceutical', 'Public Utility', 'Investment And Financing', 'Transport', 'Healthcare', 'Food Segmentation', 'Real Estate Management And Development', 'Media And Entertainment', 'Real Estate Traded Funds', 'Telecommunications', 'Applications And Technology Services', 'Insurance']


In [14]:
unique_sectors_en = ['Banks', 'Tasi Main Market Index', 
                     'Basic Materials', 'Capital Goods', 
                     'Long-Term Goods', 'Retail Of Luxury Goods', 
                     'Consumer Services', 'Commercial And Professional Services', 
                     'Energy', 'Food Production', 'Pharmaceutical', 
                     'Public Utility', 'Investment And Financing', 'Transport', 
                     'Healthcare', 'Food Segmentation', 'Real Estate M & D', 
                     'Media And Entertainment', 'Real Estate Traded Funds', 
                     'Telecommunications', 'Applications And Technology Services', 'Insurance']


sector_map = {sector_ar:sector_en for sector_ar, sector_en in zip(unique_sectors, unique_sectors_en)}

In [15]:
df['Sector_en'] = df['Sector'].map(sector_map)

In [16]:
df.groupby(['Sector', 'Sector_en']).size()

Sector                       Sector_en                           
إدارة وتطوير العقارات        Real Estate M & D                       1961
إنتاج الأغذية                Food Production                         2423
الإتصالات                    Telecommunications                      1099
الإستثمار والتمويل           Investment And Financing                 826
الإعلام والترفيه             Media And Entertainment                  416
الادوية                      Pharmaceutical                           243
البنوك                       Banks                                   1477
التأمين                      Insurance                               7643
التطبيقات وخدمات التقنية     Applications And Technology Services      22
الخدمات الإستهلاكية          Consumer Services                       1134
الخدمات التجارية والمهنية    Commercial And Professional Services     322
الرعاية الصحية               Healthcare                               887
السلع الرأسمالية             Capital Goods    

In [17]:
drop_cols = ['Date', 'Company ID']

In [18]:
df.drop(drop_cols, axis=1, inplace=True)

In [19]:
# df['Date'] = pd.to_datetime(df['Date_1st_Additional'] + ' ' + df['Time'])

In [20]:
df['Date'] = df.apply(lambda r : pd.datetime.combine(r['Date_1st_Additional'], r['Time']), axis=1)

D:\Anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.



In [21]:
df.drop(['Date_1st_Additional'], axis=1, inplace=True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34386 entries, 0 to 34385
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Sector                     34386 non-null  object        
 1   Company Name               34386 non-null  object        
 2   Time                       34386 non-null  object        
 3   News Title                 34386 non-null  object        
 4   News Details               30098 non-null  object        
 5   Polarity                   34386 non-null  object        
 6   Polarity Words             34386 non-null  object        
 7   Oppening                   34269 non-null  float64       
 8   The Highest                34269 non-null  float64       
 9   The Lowest                 34269 non-null  float64       
 10  Closing                    34269 non-null  float64       
 11  Changing                   34269 non-null  float64       
 12  Chan

In [23]:
df.columns = ['The Quantity Handled' if col=='The quantity Handled' else col for col in df.columns]

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34386 entries, 0 to 34385
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Sector                     34386 non-null  object        
 1   Company Name               34386 non-null  object        
 2   Time                       34386 non-null  object        
 3   News Title                 34386 non-null  object        
 4   News Details               30098 non-null  object        
 5   Polarity                   34386 non-null  object        
 6   Polarity Words             34386 non-null  object        
 7   Oppening                   34269 non-null  float64       
 8   The Highest                34269 non-null  float64       
 9   The Lowest                 34269 non-null  float64       
 10  Closing                    34269 non-null  float64       
 11  Changing                   34269 non-null  float64       
 12  Chan

In [25]:
# profile = ProfileReport(df, title="EDA Report", explorative=True)
# profile.to_widgets()
# profile.to_file("EDA-Report.html")

## Data Cleaning and Normalization

### Exploring Nans

In [26]:
df.isnull().sum()      # nans in each column

Sector                          0
Company Name                    0
Time                            0
News Title                      0
News Details                 4288
Polarity                        0
Polarity Words                  0
Oppening                      117
The Highest                   117
The Lowest                    117
Closing                       117
Changing                      117
Changing%                     117
The Quantity Handled          117
Total Current Value (SAR)     117
Number of Deals               117
Sector_en                       0
Date                            0
dtype: int64

In [27]:
# df[df['News Details'].isnull()]

In [28]:
# remove NANs
df = df.dropna(subset=['News Details'])

In [29]:
df.isna().sum()

Sector                         0
Company Name                   0
Time                           0
News Title                     0
News Details                   0
Polarity                       0
Polarity Words                 0
Oppening                     105
The Highest                  105
The Lowest                   105
Closing                      105
Changing                     105
Changing%                    105
The Quantity Handled         105
Total Current Value (SAR)    105
Number of Deals              105
Sector_en                      0
Date                           0
dtype: int64

### Exploring Duplicates

In [30]:
# check number of duplicated records (rows)
df.duplicated().sum()

4

In [31]:
# check the duplicated reviews
print_full(df.loc[df['News Details'].duplicated()])

,Sector,Company Name,Time,News Title,News Details,Polarity,Polarity Words,Oppening,The Highest,The Lowest,Closing,Changing,Changing%,The Quantity Handled,Total Current Value (SAR),Number of Deals,Sector_en,Date
148,البنوك,بنك الرياض,08:07:05,يدعو بنك الرياض السادة المساهمين لحضور اجتماع الجمعية العامة العادية (إعلان تذكيري),يسر مجلس إدارة بنك الرياض ، دعوة مساهميه الكرام لحضور اجتماع الجمعية العامة العادية الذي سيعقد ، بمشيئة الله ، عند الساعة السادسة من مساء يوم الاثنين 23/12/1434هـ الموافق 28/10/2013م بمقر الإدارة العامة لبنك الرياض بطريق الملك عبد العزيز حي المربع بمدينة الرياض ، وذلك للنظر في جدول الأعمال التالي: البند الوحيد :انتخاب أعضاء مجلس إدارة البنك لمدة ثلاث سنوات قادمة اعتبارا من تاريخ 26/12/1434هـ الموافق31/10/2013م من بين المرشحين لعضوية مجلس الإدارة. هذا وتتوافر البيانات عن المرشحين لعضوية مجلس الإدارة بمقر الإدارة العامة للبنك للاطلاع عليها ، خلال الخمسة و عشرون يوماً السابقة لموعد الاجتماع .ويحق لكل مساهم يمتلك عشرين سهماً على الأقل حضور الاجتماع المشار إليه ، وله أن يوكل عنه مساهماً آخراً يمتلك عشرين سهما على الأقل لتمثيله في الاجتماع بموجب التوكيل المبين نصه أدناه (وفقاً لضوابط التوكيل الصادرة بقرار معالي وزير التجارة رقم 294 في 13/2/1422هـ ). وبحيث يرسـل التوكيل أدناه إلى بنك الريــاض (لعنــاية إدارة علاقـات المساهمــين ص.ب. 22622 الرياض 11416) ليصل البنك قبل موعد الاجتماع بسبعة أيام على الأقل . وعلى كل مساهم يرغب في حضور الاجتماع إحضار بطاقة الأحوال المدنية والمستندات الدالة على ملكيته وموكليه للأسهم. ويكون اجتماع الجمعية العامة العادية صحيحاً إذا حضره مساهمون يمتلكون نصف رأس مال البنك على الأقل.ووفقا للنظام الأساسي للبنك فإن انتخاب اعضاء مجلس الإدارة سيتم وفق اسلوب التصويت العادي.ويرجى من المساهمين الكرام الحضور قبل موعد الاجتماع بساعة واحدة على الأقل ، وذلك لإنهاء إجراءات تسجيلهم.وللرد على أي استفسار ، يرجى الاتصال عبر الهاتف رقم 3030-401/011 توصيله رقم ( 2444 ، 2445 ) ، أو عبر الفاكس رقم 0016-403/011 (الرياض) خلال ساعات الدوام الرسمي للبنك أو على البريد الإلكتروني : shareholders@riyadbank.com واللـــــه الموفــــــق بنك الرياض ، المركز الرئيسي - طريق الملك عبد العزيز - ص ب : 22622 الرياض 11416 www.riyadbank.comتوكيل خاصأنا المساهم / .................................................. الموقع أدناه ، بموجــــب بطاقة الأحـــوال الـمدنيـة رقم / ........................... الصادرة من بتاريخ / / هـ وبصفتي أحد المساهمين في بنك الرياض ( شركة مساهمة سعودية )، والمالك لعدد ( ) سهم قد وكلت المساهم .......................... بطاقة الأحوال المدنية رقم / .......................... الصادرة من بتاريخ / / هـ ( وهو من غير أعضاء مجلس الإدارة أو موظفي البنك أو المكلفين بالقيام بصفة دائمة بعمل فني أو إداري لحسابه ) ، لينوب عني في حضور اجتماع الجمعية العامة العادية لمساهمي بنك الرياض والمقرر عقده بمشيئة الله، في يوم الاثنين 23/12/1434هـ الموافق 28/10/2013م، والتصويت على بنود جدول أعمال الجمعية نيابة عني والتوقيع على كل المستندات المطلوبة والضرورية واللازمة لإجراءات الاجتماع . كما يسري هذا التوكيل للاجتماع الثاني في حال تأجيل الاجتماع . تحريرا في : / / 1434هـالموافق : / / 2013مالأسم :التوقيع : المصادقة على صحة التوقيع:الصفة :,Positive,توقيع - عقد,14.10,14.25,14.05,14.20,0.10,0.71,670654.0,9.484389e+06,114.0,Banks,2013-10-21 08:07:05
262,البنوك,البنك السعودي الفرنسي,15:42:37,يدعو البنك السعودي الفرنسي مساهميه لحضور اجتماع الجمعية العامة العادية,يسر مجلس إدارة البنك السعودي الفرنسي دعوة مساهميه الكرام لحضور اجتماع الجمعية العامة العادية والتي ستعقد بإذن الله في تمام الساعة السادسة والنصف بعد صلاة المغرب ليوم الأحد 22/4/1432هـ الموافق (27/3/2011) بالدور الثامــن بمقر المركز الرئيسي للبنك الكائن بشارع المعذر بمدينة الرياض وذلك للنظر في جدول الأعمال التالي :- (1) الموافقة على تقرير مجلس الإدارة عن العام المالي المنتهي في 31/12/2010م. (2) التصديق على الميزانية العامة للبنك كما في 31/12/2010م وعلى حساب الأرباح والخسائر عن نفس السنة المالية. (3) الموافقة على تقرير مراقبي الحسابات عن العام المالي المنتهي في 31/12/2010م. (4) الموافقة على توصية مجلس الإدارة بشأن توزيع الأرباح عن العـــام المالي 2010م بمقدار (ريال واحد) للسهم الواحد آي بنسبة 10% من القيمة الاسمية للسهم لحملة أسهم البنــــــك المسجلين بسجلات مساهمي البنك كما في نهاية تداو

On close examination of the duplicated rows, it turned out they're not fully duplicated. So we'll leave them as is 

### Storing the Clean Data for Later Use

In [32]:
df.to_csv('data/clean_data.csv', index=False)

In [33]:
reset()

In [34]:
df1.head(2)

NameError: name 'df1' is not defined

## Descriptive Statistics

In [ ]:
cats = ['Sector', 'Company Name', 'Polarity']
nums = ['Oppening', 'The Highest', 'The Lowest', 'Closing', 'Changing', 'Changing%', 
        'The Quantity Handled', 'Total Current Value (SAR)', 'Number of Deals']

texts = ['News Title', 'News Details', 'Polarity Words']

date = ['Date']

#### Numerical Variables Statistics

**Note:** The huge numbers in the column `Total Current Value` is due to the huge amount itself. it's not an approximation

In [ ]:
df[nums].describe().round(2)

In [ ]:
def describe_cat(cat_var, n=5):
    if cat_var == 'Date':
        col = df[cat_var].dt.date
    else:
        col = df[cat_var]
        
    unique = len(col.unique())
    top_counts = col.value_counts().to_frame().reset_index().head(n)
    top_counts_str = "، ".join([f'{level}: {count}' for level,count in top_counts.values])
    
    summary = pd.Series({'Variable': cat_var, 'Unique': unique, 'Top Counts': top_counts_str})
    
    return summary
    
    
def describe_cats(cat_vars):
    summary = pd.DataFrame(np.nan, columns=['Variable', 'Unique', 'Top Counts'], index=range(len(cat_vars)))
    
    for i, cat in enumerate(cat_vars):
        summary.iloc[i] = describe_cat(cat)
    
    summary['Unique'] = summary['Unique'].astype('int')
        
    return summary

In [ ]:
describe_cat('Date')

In [ ]:
print_full(describe_cats(cats+date))

In [ ]:
reset()

## Exploratory Data Analysis

### Exploring Singe Categorical Variables

#### The Polarity

In [ ]:
# !pip install notebook ipywidgets

In [ ]:
from IPython.display import HTML

In [35]:
fig = (  df['Polarity']
       .value_counts()
       .reset_index()
       .pipe(px.pie,
             values='Polarity', names='index',
             title='Polarity Distribution',
             color='Polarity',
             color_discrete_sequence=px.colors.qualitative.Prism,
             hole=0.1)
       .update_traces(textinfo='percent+label+value')
    
)

HTML(fig.to_html())

#### Most Featured Sectors

In [43]:
(df
 .groupby('Sector_en')
 .size()
 .to_frame(name='Count')
 .reset_index()
 .sort_values('Count', ascending=False)
 .head(n_most)
 .pipe(px.bar,
       x='Sector_en', y='Count', 
#        color='Count',
#        color_continuous_scale=px.colors.sequential.GnBu,
#        color_continuous_scale=px.colors.sequential.Blues,
#        color_continuous_scale=px.colors.sequential.Plasma_r,
       text='Count', title='Most Featured Sectors', labels={'Count': 'No. Articles', 'Sector_en':'Sector'}))

In [44]:
featured_sectors = (df['Sector_en']
                     .value_counts()
                     .head(n_most)
                     .index.tolist())
featured_sectors

['Insurance',
 'Basic Materials',
 'Capital Goods',
 'Food Production',
 'Real Estate M & D',
 'Banks',
 'Retail Of Luxury Goods',
 'Telecommunications',
 'Consumer Services',
 'Transport']

#### Most Featured Companies

In [45]:
n_most = 10

(df['Company Name']
 .value_counts()
 .head(n_most)
 .iplot(kind='bar', color='navy', yTitle='# of News Articles', title=f'Most {n_most} Featured Companies'))

In [46]:
featured_companies = (df['Company Name']
                     .value_counts()
                     .head(n_most)
                     .index.tolist())
featured_companies

['شركة أبناء عبدالله عبدالمحسن الخضري',
 'شركة إتحاد عذيب للإتصالات',
 'شركة الباحة للإستثمار والتنمية',
 'الشركة السعودية الهندية للتأمين التعاوني',
 'شركة طيبة للإستثمار',
 'شركة الكابلات السعودية',
 'الشركة الوطنية السعودية للنقل البحري',
 'شركة التعدين العربية السعودية',
 'شركة الأحساء للتنمية',
 'الشركة السعودية للنقل الجماعي']

### Exploring Interaction Between 2 Categorical Variables

#### Polarity & Sector

##### Method (1) Grouped Barplot

In [47]:
polarity_by_sector = (df
                      .groupby(['Sector_en', 'Polarity'])
                      .size()
                      .unstack()
                      .fillna(0)
                      .sort_values('Positive', ascending=False))

polarity_by_sector

Polarity,Negative,Neutral,Positive
Sector_en,,,
Basic Materials,586,2239,3391
Insurance,1167,2601,2985
Capital Goods,293,988,1314
Real Estate M & D,103,635,990
Food Production,237,908,981
Banks,37,416,799
Consumer Services,48,285,654
Retail Of Luxury Goods,46,338,651
Transport,32,364,514


In [48]:
polarity_by_sector.loc[featured_sectors]

Polarity,Negative,Neutral,Positive
Sector_en,,,
Insurance,1167,2601,2985
Basic Materials,586,2239,3391
Capital Goods,293,988,1314
Food Production,237,908,981
Real Estate M & D,103,635,990
Banks,37,416,799
Retail Of Luxury Goods,46,338,651
Telecommunications,159,419,423
Consumer Services,48,285,654


In [49]:
(polarity_by_sector.loc[featured_sectors]
 .iplot(kind='bar',
        colorscale='-gnbu',#'set1',
        layout=dict(title='Distribution of Polarity over Sectors',
                    yaxis=dict(title='# of Articles'),
                    xaxis=dict(title='Sector'))))

In [50]:
(polarity_by_sector.loc[featured_sectors]
 .reset_index()
 .melt(id_vars='Sector_en', value_name='Count')
 .pipe(
     px.bar,
     x='Sector_en', y='Count', color='Polarity', text='Count',
     barmode='group', opacity=0.8,
     title='Distribution of Polarity over Sectors',
     labels={'Count': 'No. of Articles'},
     color_discrete_sequence = px.colors.diverging.Spectral_r,
#      color_discrete_sequence=px.colors.sequential.Plasma,
#      color_discrete_sequence=px.colors.qualitative.Prism,
#      color_discrete_sequence=px.colors.sequential.GnBu_r,
#      color_discrete_sequence=px.colors.qualitative.Set1
))

##### Method (2) Heatmap

In [ ]:
(polarity_by_sector.loc[featured_sectors]
 .iplot(kind='heatmap',
        title='Distribution of Polarity over Sectors',
        colorscale='gnbu'))

In [ ]:
(polarity_by_sector.loc[featured_sectors]
 .T
 .pipe(px.imshow,
       color_continuous_scale=px.colors.sequential.GnBu,
       title='Distribution of Polarity over Sector'))

#### Polarity & Company

##### Method (1) Groupded Barplot

In [ ]:
polarity_by_company = (df
                      .groupby(['Company Name', 'Polarity'])
                      .size()
                      .unstack()
                      .fillna(0)
                      .sort_values('Positive', ascending=False))

polarity_by_company

In [ ]:
(polarity_by_company.loc[featured_companies]
 .iplot(kind='bar',
        colorscale='blues',
        layout=dict(title='Distribution of Polarity over Companies',
                    yaxis=dict(title='# of Articles'),
                    xaxis=dict(title='Company'))))

##### Method (2) Heatmap

In [ ]:
(polarity_by_company.loc[featured_companies]
 .iplot(kind='heatmap',
        title='Distribution of Polarity over Companies',
        colorscale='blues'))

### Exploring Interaction Between 3 Categorical Variables

In [ ]:
df_featured = df[(df['Company Name'].isin(featured_companies)) & (df['Sector_en'].isin(featured_sectors))]

In [ ]:
px.parallel_categories(df_featured, width=1000, height=700, title='Parallel Plot of Sector, Company, Polarity',
                       labels={'Sector_en': 'Sector'})

### Exploring Single Numerical Variables

In [ ]:
df['Closing'].iplot(kind='box', 
                    boxpoints='outliers',
                    color='blue',
                    title='Distribution of Closing Price')

In [ ]:
df.pipe(px.box,
        y='Closing',
        points='outliers',
        title='Distribution of Closing Price')

In [ ]:
df['Changing%'].iplot(kind='box', 
                      boxpoints='outliers',
                      color='purple',
                      title='Distribution of Changing%')

In [ ]:
(df
 .pipe(px.box,
       y='Changing%'))

### Qustion: How wast the variable `Changing%` computed?

In [ ]:
df['change_pct'] = df['Closing'].pct_change()

In [ ]:
df[['Changing%', 'change_pct']]

### Exploring Relationship Between Numerical Variables

In [ ]:
df.columns

In [ ]:
(df[nums]
 .corr()
 .fillna(0)
 .round(2))

In [ ]:
(df[nums]
 .corr()
 .iplot(kind='heatmap',
        colorscale='ylgnbu',#'rdylbu',#,rdbu, 
        layout=dict(title='Correlation Between All Numerical Variables',
                    width=700,
                    margin=dict(b=120))))

#### Number of Deals vs. Changing

In [ ]:
(df.fillna(0)
 .pipe(px.scatter,
       size='Total Current Value (SAR)',
       size_max=70,
       x='Number of Deals',
       y='Changing',
       title='<b>Number of Deals vs. Changing in Price</b> <br>size deontes <i>Total Current Value</i>',
       range_y=[-20, 20],
       range_x=[0, 25000]))

In [ ]:
fig = (df.fillna(0)
         .pipe(px.scatter,
               x='Closing',
               y='Changing',
               color='Polarity',
               size='Total Current Value (SAR)',
               size_max=60,
               marginal_x='histogram',
               color_discrete_sequence = px.colors.diverging.Spectral_r,
#                color_discrete_sequence=px.colors.sequential.GnBu,
#                color_discrete_sequence=['red', 'blue', 'green'],
               range_y=[-15, 15],
               range_x=[0, 200],
               height=600,
               facet_col='Polarity',
               title='<b>Closing Price vs. Changing in Price Across Polarities</b><br>size deontes <i>Total Current Value</i>'))

fig.for_each_annotation(lambda a: a.update(text=''))
fig.show()

In [ ]:
fig = (df.fillna(0)
        .pipe(px.scatter,
              x='Closing',
              y='Changing%',
              color='Polarity',
              size='Total Current Value (SAR)',
              size_max=60,
              marginal_x='histogram',
             color_discrete_sequence = px.colors.diverging.Spectral_r,
#             color_discrete_sequence=px.colors.sequential.GnBu_r,
#             color_discrete_sequence = px.colors.diverging.Spectral_r,
#             color_discrete_sequence=['red', 'blue', 'green'],
             range_y=[-20, 20],
             range_x=[0, 300],
             height=600,
             facet_col='Polarity',
             title='<b>Closing Price vs. Changing Percent in Price</b><br>size deontes <i>Total Current Value</i>'))

fig.for_each_annotation(lambda a: a.update(text=''))
fig.show()

### Exploring Interaction Between Numerical and Categorical

In [ ]:
returns_by_polarity_stats = (df.reset_index()
                             .pivot(columns='Polarity', values='Changing%')).describe()

returns_by_polarity_stats

In [ ]:
median = (df[['Polarity', 'Changing%']].reset_index()
          .pivot(columns='Polarity', values='Changing%')
          .apply(lambda x: x.median()))

median.name = 'median'

In [ ]:
returns_by_polarity_stats.append(median)

In [ ]:
(df.reset_index()
 .pivot(columns='Polarity', values='Changing%')
 .iplot(kind='box', boxpoints='outliers',
        colorscale='gnbu',
        layout=dict(title='Distribution of Percent Change Over Polarity')))

In [ ]:
(df
 .pipe(px.box,
       x='Polarity',
       y='Changing%',
       color='Polarity',
       color_discrete_sequence = px.colors.diverging.Spectral_r,
       points='outliers',
       title='Distribution of Percent Change Over Polarity'))

# fig.update_layout(showlegend=False)

From the Boxplot above we can't deduce a strong correlation between the polarity of a news article and a change in stock price. 

This should be corroborated by a statistical test, though!

### What's Happening over Time? 

#### Exploring the Date Variable

In [ ]:
df.set_index('Date', inplace=True)

In [ ]:
df = df.sort_index()

In [ ]:
date_hist = dict(
    data=[
        Histogram(
            x=df.index,
        marker=dict(
            opacity=0.9,
            line=dict(
                color='white', width=1
            )
         )
      )
    ],
    layout=dict(
        title='Distribution of No. of Articles over Months',
        xaxis=dict(
            range=[df.index.min() - pd.Timedelta(days=90), df.index.max()]
        )
    )
)

pyo.iplot(date_hist)

## The Following is Additional Extra Work

#### Moving Avg. of Numerical Variables 

In [ ]:
company_name = 'شركة أبناء عبدالله عبدالمحسن الخضري'
window = 10
num_var = 'Closing'

fig = (df
        .loc[df['Company Name'] == company_name, num_var]
        .sort_index()
        .rolling(window).mean()
        .iplot(mode='lines', color='blue', size=8, asFigure=True, 
               title=f'{window} Days Moving Avg. ({num_var}) <br> {company_name}'))

In [ ]:
fig

In [ ]:
company_name = 'شركة أبناء عبدالله عبدالمحسن الخضري'

fig_1 = (df
 .loc[df['Company Name'] == company_name]
 .groupby('Polarity')
 .resample('A')
 .size()
 .T
 .iplot(kind='bar', barmode='group',
        colorscale='gnbu',
        title=f'Polarity Evolution over Time <br> {company_name}',
        asFigure=True))

In [ ]:
fig_1

In [ ]:
fig_1['layout']['title']['text']

In [ ]:
fig_1.add_trace(fig['data'][0])


## Market Index

We can create an index for the market by choosing the largest company in each sector, in terms of `Total Current Value`. Then plot the overall index market cap over time overlayed on the sentiment as above

In [ ]:

index_comps = df.loc['2019'].set_index('Company Name').groupby('Sector_en')['Total Current Value (SAR)'].nlargest(1)

index_comps = index_comps.reset_index().sort_values('Total Current Value (SAR)', ascending=False)

index_comps

In [ ]:
index_comps_names = index_comps['Company Name']; index_comps_names

In [ ]:
df_index = df[df['Company Name'].isin(index_comps_names)]

In [ ]:
index_series = (df_index[['Company Name', 'Total Current Value (SAR)']]
                 .pivot(columns='Company Name', values='Total Current Value (SAR)')
                 .sum(axis=1)
                 .loc[: 'Nov 2019'])

In [ ]:
(df_index[['Company Name', 'Total Current Value (SAR)']]
 .pivot(columns='Company Name', values='Total Current Value (SAR)')
 .sum(axis=1)
 .loc['2011': 'Nov 2019']
 .rolling(180).mean()
 .iplot(color='blue',
        layout=dict(title='30-days Moving Average of Market Index')))

In [ ]:
index_series.iplot()

In [ ]:
(df_index
 .groupby('Polarity')
 .resample('A')
 .size()
 .T
 .iplot(kind='bar', title='Annual Polarity Distribution for the Market Index'))

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1)

In [ ]:
with open('ArSEL1.0.txt') as f:
    text = f.read()
    text = re.sub('###', ';', text)


In [ ]:
print(text[:2000])

In [ ]:
with open('ArSEL1.0_modified.txt', 'w') as f:
    f.write(text)

In [ ]:
cols = 'AWN_OFFSET;EWN_OFFSET;POS_Tag;AWN_Lemma;SAMA_Lemma;Pos_Sentiment_score;Neg_Sentiment_Score;Confidence;AFRAID;AMUSED;ANGRY;ANNOYED;DONT_CARE;HAPPY;INSPIRED;SAD'.split(';')

In [ ]:
cols

In [ ]:
lex = pd.read_table('ArSEL1.0_modified.txt', sep=';', skiprows=20, header=None)
lex.columns = cols

In [ ]:
lex.head()

In [ ]:
lex.info()

In [ ]:
lex['SAMA_Lemma'].value_counts()

### How to Read Facebook's Fasttext pretrained Models into Gensim

read this ASAP: <a href='https://radimrehurek.com/gensim/models/fasttext.html' target='_blank'>fasttext loading into gensim</a>